In [1]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
import numpy as np
import pymysql

In [2]:
#conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='101Monkeybob!',db='Final_Project')
#df=pd.read_sql_query("SELECT * FROM 'house' ",conn)


In [3]:
# Import Clean House Data
house_data = pd.read_csv("clean_house_data_ML.csv",low_memory = False)
imp_df = pd.DataFrame(house_data)
imp_df.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,PRICE,INDEX_BIN
0,3,1,8,3,2010.0,1.0,0,1424,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,1325000.0,52
1,1,0,6,3,2006.0,1.0,0,2090,20009.0,2,7.875,0.0,1.0,0.0,1.0,0.0,0.0,33232.0,1
2,2,1,5,3,2012.0,1.0,1,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,907400.0,36
3,2,1,8,3,2012.0,1.0,2,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,1065000.0,42
4,2,1,7,2,2009.0,1.0,1,838,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,216000.0,8


In [4]:
index = imp_df[imp_df.PRICE >= 1000000].index
index2 = imp_df[imp_df.PRICE <= 100000].index
imp_df = imp_df.drop(index)
imp_df = imp_df.drop(index2)

In [5]:
#Binning Price data
bins = [100000, 150000, 200000, 250000,
        300000,350000, 400000,450000,500000,
        550000,600000,650000,700000,750000,
        800000,850000,900000,950000,1000000,
        1050000,1100000,1150000, 1200000,]
labels = [x*1 for x in range(1,23)]
cut = pd.cut(imp_df['PRICE'], bins=bins, labels=labels)
cut_df = pd.DataFrame(cut)
cut_df['REAL_PRICE'] = imp_df.PRICE
cut_df = cut_df.rename(columns={'PRICE':'INDEX_BIN'})
cut_df.head(50)

,INDEX_BIN,REAL_PRICE
2,17,907400.0
4,3,216000.0
5,18,990000.0
6,12,682500.0
7,18,993500.0
8,13,745000.0
9,4,277000.0
11,16,900000.0
12,6,360000.0
14,14,770000.0


In [6]:
imp_df=imp_df.drop("INDEX_BIN", axis = 1)
imp_df=imp_df.drop("PRICE", axis = 1)
imp_df

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
2,2,1,5,3,2012.0,1.0,1,2090,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0
4,2,1,7,2,2009.0,1.0,1,838,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0
5,2,1,6,2,2005.0,1.0,0,1700,20009.0,2,6.375000,0.0,1.0,0.0,1.0,0.0,0.0
6,2,1,7,3,2014.0,1.0,2,1000,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0
7,1,1,6,3,2005.0,1.0,0,1424,20009.0,2,6.375000,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37012,2,1,4,3,2014.0,1.0,0,1733,20032.0,8,3.250000,0.0,1.0,0.0,0.0,1.0,0.0
37013,2,1,7,3,2008.0,1.0,1,1734,20032.0,8,4.928571,0.0,1.0,0.0,0.0,1.0,0.0
37014,1,1,9,3,2001.0,1.0,0,2799,20032.0,8,6.772727,0.0,1.0,0.0,0.0,1.0,0.0
37016,1,1,7,3,2012.0,1.0,0,1919,20032.0,8,3.250000,0.0,1.0,0.0,0.0,1.0,0.0


In [7]:
imp_df = pd.merge(imp_df, cut_df, left_index=True, right_index=True)
imp_df.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,INDEX_BIN,REAL_PRICE
2,2,1,5,3,2012.0,1.0,1,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,17,907400.0
4,2,1,7,2,2009.0,1.0,1,838,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,3,216000.0
5,2,1,6,2,2005.0,1.0,0,1700,20009.0,2,6.375,0.0,1.0,0.0,1.0,0.0,0.0,18,990000.0
6,2,1,7,3,2014.0,1.0,2,1000,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,12,682500.0
7,1,1,6,3,2005.0,1.0,0,1424,20009.0,2,6.375,0.0,1.0,0.0,1.0,0.0,0.0,18,993500.0


In [8]:
imp_df = imp_df.drop(columns = ['REAL_PRICE'])

In [9]:
imp_df.INDEX_BIN = imp_df.INDEX_BIN.astype('int32')

In [10]:
# Define the features 
X = imp_df.copy()
X = X.drop("INDEX_BIN", axis = 1)
X.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
2,2,1,5,3,2012.0,1.0,1,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
4,2,1,7,2,2009.0,1.0,1,838,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
5,2,1,6,2,2005.0,1.0,0,1700,20009.0,2,6.375,0.0,1.0,0.0,1.0,0.0,0.0
6,2,1,7,3,2014.0,1.0,2,1000,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
7,1,1,6,3,2005.0,1.0,0,1424,20009.0,2,6.375,0.0,1.0,0.0,1.0,0.0,0.0


In [11]:
# Define target set
y = imp_df['INDEX_BIN'].ravel()
y[:5]

array([17,  3, 18, 12, 18])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [23]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators = 1000, random_state = 2)
model.fit(X_train,y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=2)

In [24]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
accuracy_score = balanced_accuracy_score(y_test, y_pred)
accuracy_score

0.0867016607893201

In [25]:
#Fitting the model
confusion_matrix(y_test,y_pred)

array([[65, 58, 44, 48, 43, 42, 50, 34, 32, 35, 33, 26, 17, 21, 19, 25,
        29, 23],
       [58, 63, 47, 36, 29, 35, 30, 19, 26, 30, 30, 24, 27, 21, 20, 20,
        20, 22],
       [54, 53, 52, 23, 26, 26, 30, 35, 22, 12, 24, 15, 18, 34, 24, 33,
        27, 38],
       [44, 68, 45, 41, 20, 34, 32, 29, 32, 28, 17, 10, 20, 25, 26, 20,
        38, 32],
       [38, 51, 40, 29, 31, 31, 33, 30, 22, 25, 21, 34, 22, 23, 31, 29,
        53, 42],
       [35, 46, 35, 35, 45, 40, 49, 37, 29, 37, 33, 33, 24, 27, 26, 37,
        39, 38],
       [42, 46, 35, 23, 34, 30, 41, 25, 20, 39, 20, 20, 29, 26, 25, 18,
        39, 33],
       [32, 35, 18, 24, 20, 19, 40, 28, 27, 30, 26, 23, 27, 29, 25, 27,
        41, 37],
       [19, 23, 17, 25, 14, 22, 32, 27, 31, 29, 22, 20, 26, 22, 25, 29,
        33, 39],
       [17, 26, 23, 10, 17, 18, 25, 23, 30, 29, 28, 30, 22, 25, 27, 23,
        36, 48],
       [19, 17, 15, 19,  9, 20, 17, 19, 24, 22, 28, 15, 27, 22, 23, 33,
        27, 41],
       [19, 14,  9, 1

In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.14      0.10      0.94      0.12      0.31      0.09       644
          2       0.12      0.11      0.94      0.12      0.33      0.10       557
          3       0.12      0.10      0.95      0.11      0.30      0.08       546
          4       0.11      0.07      0.96      0.09      0.26      0.06       561
          5       0.09      0.05      0.96      0.07      0.23      0.05       585
          6       0.10      0.06      0.95      0.08      0.24      0.05       645
          7       0.09      0.08      0.95      0.08      0.27      0.06       545
          8       0.07      0.06      0.95      0.06      0.23      0.05       508
          9       0.08      0.07      0.95      0.07      0.25      0.06       455
         10       0.07      0.06      0.95      0.07      0.25      0.05       457
         11       0.07      0.07      0.95      0.07      0.26      0.06       397
   

In [30]:
# List the features sorted in descending order by feature importance
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.28475470496005456, 'LANDAREA'),
 (0.14560425867782226, 'SALEDATE'),
 (0.10735583401942995, 'ROOMS'),
 (0.10028923032060788, 'PRIME'),
 (0.06303877151427884, 'BEDRM'),
 (0.06263808217936916, 'BATHRM'),
 (0.0584158720107387, 'HF_BATHRM'),
 (0.056621877707774, 'ZIPCODE'),
 (0.03705235168586852, 'FIREPLACES'),
 (0.03570361447092813, 'WARD'),
 (0.017023295225767184, 'AC_Y'),
 (0.016972070464402196, 'AC_N'),
 (0.004711822749504939, 'QUADRANT_SE'),
 (0.004642714170109279, 'QUADRANT_NE'),
 (0.0041317739675861016, 'QUADRANT_NW'),
 (0.0010437258757582705, 'QUADRANT_SW'),
 (0.0, 'KITCHENS')]